In [ ]:
import pandas as pd
import json

INPUT_FILE = 'terrorism_dataset.csv' 
OUTPUT_JSON = 'target_radial_tree.json'
TOP_K_SUBTYPES = 5

TARGET_MAP = {
    'Military': 'military_police',
    'Police': 'military_police',
    'Government (General)': 'government',
    'Government (Diplomatic)': 'government',
    'Business': 'business',
    'Private Citizens & Property': 'citizens',
    'Transportation': 'transportations'
}

ORDERED_KEYS = ['military_police', 'government', 'business', 'citizens', 'transportations']

def prepare_data():
    print("Loading data...")
    try:
        cols = ['targtype1_txt', 'targsubtype1_txt']
        df = pd.read_csv(INPUT_FILE, encoding='ISO-8859-1', usecols=cols, low_memory=False)
    except FileNotFoundError:
        print(f"Error: file not found.")
        return

    df = df[df['targtype1_txt'].isin(TARGET_MAP.keys())].copy()
    df['category'] = df['targtype1_txt'].map(TARGET_MAP)
    df['targsubtype1_txt'] = df['targsubtype1_txt'].fillna('Unknown')

    total_attacks = len(df)
    cat_counts = df['category'].value_counts().to_dict()
    
    root_node = {
        "name": "ALL ATTACKS",
        "value": total_attacks,
        "type": "root",
        "children": []
    }

    for cat_key in ORDERED_KEYS:
        count = cat_counts.get(cat_key, 0)
        
        cat_node = {
            "name": cat_key,
            "value": count,
            "type": "category",
            "children": []
        }

        sub_df = df[df['category'] == cat_key]
        sub_counts = sub_df['targsubtype1_txt'].value_counts().head(TOP_K_SUBTYPES)
        
        for sub_name, sub_val in sub_counts.items():
            cat_node["children"].append({
                "name": sub_name,
                "value": int(sub_val),
                "type": "subtype"
            })
        
        root_node["children"].append(cat_node)

    final_json = {}
    for key in ORDERED_KEYS:
        final_json[key] = root_node

    with open(OUTPUT_JSON, 'w') as f:
        json.dump(final_json, f, indent=2)
    print("Done.")

if __name__ == "__main__":
    prepare_data()